<a href="https://colab.research.google.com/github/pellinuz/vms/blob/master/enigmacracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Nota:** Assicurati di aver caricato il tuo file `.env` nella sessione di Colab in modo che lo script possa accedervi. Puoi caricarlo usando l'icona della cartella a sinistra.

In [39]:
# EnigmaCracker
import subprocess
import sys
import os
import platform
import requests
import logging
import time
from dotenv import load_dotenv
from bip_utils import (
    Bip39MnemonicGenerator,
    Bip39SeedGenerator,
    Bip44,
    Bip44Coins,
    Bip44Changes,
    Bip39WordsNum,
)

# Constants
LOG_FILE_NAME = "enigmacracker.log"
ENV_FILE_NAME = "EnigmaCracker.env"
WALLETS_FILE_NAME = "wallets_with_balance.txt"

# Global counter for the number of wallets scanned
wallets_scanned = 0

# Get the absolute path of the directory where the script is located
# In Colab, /content/ is the working directory for executed scripts, so we can use it directly.
directory = "/content/"
# Initialize directory paths
log_file_path = os.path.join(directory, LOG_FILE_NAME)
env_file_path = os.path.join(directory, ENV_FILE_NAME)
wallets_file_path = os.path.join(directory, WALLETS_FILE_NAME)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file_path),  # Log to a file
        logging.StreamHandler(sys.stdout),  # Log to standard output
    ],
)

# Load environment variables from .env file
load_dotenv(env_file_path)

# Environment variable validation
required_env_vars = ["ETHERSCAN_API_KEY"]
missing_vars = [var for var in required_env_vars if not os.getenv(var)]
if missing_vars:
    # Log and raise error for missing environment variables
    print(f"Missing environment variables: {', '.join(missing_vars)}")
    logging.error(f"Missing environment variables: {', '.join(missing_vars)}")
    raise EnvironmentError(f"Missing environment variables: {', '.join(missing_vars)}")

# The part of the script that tried to open a new terminal is removed as it's not compatible with Colab.
# The script will now run directly in the current Colab execution environment.

def update_cmd_title():
    # This function is for updating CMD title on Windows, not applicable in Colab shell.
    # We will keep it but it won't have effect.
    if platform.system() == "Windows":
        os.system(f"title EnigmaCracker.py - Wallets Scanned: {wallets_scanned}")


def bip():
    # Generate a 12-word BIP39 mnemonic
    return Bip39MnemonicGenerator().FromWordsNumber(Bip39WordsNum.WORDS_NUM_12)


def bip44_ETH_wallet_from_seed(seed):
    # Generate an Ethereum wallet from a BIP39 seed.

    # Generate the seed from the mnemonic
    seed_bytes = Bip39SeedGenerator(seed).Generate()

    # Create a Bip44 object for Ethereum derivation
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.ETHEREUM)

    # Derive the account 0, change 0, address_index 0 path (m/44'/60'/0'/0/0)
    bip44_acc_ctx = (
        bip44_mst_ctx.Purpose()
        .Coin()
        .Account(0)
        .Change(Bip44Changes.CHAIN_EXT)
        .AddressIndex(0)
    )

    # Get the Ethereum address
    eth_address = bip44_acc_ctx.PublicKey().ToAddress()

    return eth_address


def bip44_BTC_seed_to_address(seed):
    # Generate the seed from the mnemonic
    seed_bytes = Bip39SeedGenerator(seed).Generate()

    # Generate the Bip44 object
    bip44_mst_ctx = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)

    # Generate the Bip44 address (account 0, change 0, address 0)
    bip44_acc_ctx = bip44_mst_ctx.Purpose().Coin().Account(0)
    bip44_chg_ctx = bip44_acc_ctx.Change(Bip44Changes.CHAIN_EXT)
    bip44_addr_ctx = bip44_chg_ctx.AddressIndex(0)

    # Print the address
    return bip44_addr_ctx.PublicKey().ToAddress()


def check_ETH_balance(address, etherscan_api_key, retries=3, delay=5):
    # Etherscan API endpoint to check the balance of an address
    api_url = f"https://api.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={etherscan_api_key}"

    for attempt in range(retries):
        try:
            # Make a request to the Etherscan API
            response = requests.get(api_url)
            data = response.json()

            # Check if the request was successful
            if data["status"] == "1":
                # Convert Wei to Ether (1 Ether = 10^18 Wei)
                balance = int(data["result"]) / 1e18
                return balance
            else:
                print(f"Error getting balance: {data['message']}")
                logging.error("Error getting balance: %s", data["message"])
                return 0
        except Exception as e:
            if attempt < retries - 1:
                print(f"Error checking balance, retrying in {delay} seconds: {str(e)}")
                logging.error(
                    f"Error checking balance, retrying in {delay} seconds: {str(e)}"
                )
                time.sleep(delay)
            else:
                print(f"Error checking balance: {str(e)}")
                logging.error("Error checking balance: %s", str(e))
                return 0


def check_BTC_balance(address, retries=3, delay=5):
    # Check the balance of the address
    for attempt in range(retries):
        try:
            response = requests.get(f"https://blockchain.info/balance?active={address}")
            data = response.json()
            balance = data[address]["final_balance"]
            return balance / 100000000  # Convert satoshi to bitcoin
        except Exception as e:
            if attempt < retries - 1:
                print(f"Error checking balance, retrying in {delay} seconds: {str(e)}")
                logging.error(
                    f"Error checking balance, retrying in {delay} seconds: {str(e)}"
                )
                time.sleep(delay)
            else:
                print(f"Error checking balance: {str(e)}")
                logging.error("Error checking balance: %s", str(e))
                return 0


def write_to_file(seed, BTC_address, BTC_balance, ETH_address, ETH_balance):
    # Write the seed, address, and balance to a file in the script's directory
    with open(wallets_file_path, "a") as f:
        print(f"Seed: {seed}\nAddress: {BTC_address}\nBalance: {BTC_balance} BTC\n\nEthereum Address: {ETH_address}\nBalance: {ETH_balance} ETH\n\n")
        log_message = f"Seed: {seed}\nAddress: {BTC_address}\nBalance: {BTC_balance} BTC\n\nEthereum Address: {ETH_address}\nBalance: {ETH_balance} ETH\n\n"
        f.write(log_message)
        logging.info(f"Written to file: {log_message}")


def main():
    global wallets_scanned
    try:
        while True:
            seed = bip()
            # BTC
            BTC_address = bip44_BTC_seed_to_address(seed)
            BTC_balance = check_BTC_balance(BTC_address)

            print(f"Seed:{seed}")
            print(f"BTC address: {BTC_address}")
            print(f"BTC balance: {BTC_balance} BTC")
            print("")

            logging.info(f"Seed: {seed}")
            logging.info(f"BTC address: {BTC_address}")
            logging.info(f"BTC balance: {BTC_balance} BTC")
            logging.info("")

            # ETH
            ETH_address = bip44_ETH_wallet_from_seed(seed)
            etherscan_api_key = os.getenv("ETHERSCAN_API_KEY")
            if not etherscan_api_key:
                raise ValueError(
                    "The Etherscan API key must be set in the environment variables."
                )
            ETH_balance = check_ETH_balance(ETH_address, etherscan_api_key)

            print(f"ETH address: {ETH_address}")
            print(f"ETH balance: {ETH_balance} ETH")

            logging.info(f"ETH address: {ETH_address}")
            logging.info(f"ETH balance: {ETH_balance} ETH")

            # Increment the counter and update the CMD title
            wallets_scanned += 1
            # update_cmd_title() # This line is commented out as update_cmd_title is not applicable in Colab shell

            # Check if the address has a balance
            if BTC_balance > 0 or ETH_balance > 0:
                print(f"(!) Wallet with balance found!")
                logging.info("(!) Wallet with balance found!")
                write_to_file(seed, BTC_address, BTC_balance, ETH_address, ETH_balance)

    except KeyboardInterrupt:
        print("Program interrupted by user. Exiting...")
        logging.info("Program interrupted by user. Exiting...")


if __name__ == "__main__":
    main()

Seed:buyer void example census rigid trim brick crucial uphold echo model start
BTC address: 19W98RvSAbn5ntLeNSWVxSjmwcRAqG2nBv
BTC balance: 0.0 BTC

ETH address: 0xd703680b7c7Cd0aDA90973809D2106c6FD6a74f5
ETH balance: 0.0 ETH
Seed:biology today agent snake inside culture harvest cover income type just dismiss
BTC address: 17BLUa5dnQzRBW2bDpZmNASbJa5gGWfiKp
BTC balance: 0.0 BTC

ETH address: 0x241149B2EebDcbBb0Fa372FDa8f1B64Ef7Ca9EbE
ETH balance: 0.0 ETH
Seed:expose patch prefer borrow side food major combine settle remove palace use
BTC address: 1FLRZ8kme4762cqkFzxsK61uZVLpJZqiCY
BTC balance: 0.0 BTC

ETH address: 0xfC7f6a81ea266FCaC3b8398157EaB5b65c33B9c2
ETH balance: 0.0 ETH
Seed:combine lion pizza source muscle flip file method chaos attack flush lava
BTC address: 1GUBx4Awm5VRcL7aTLron42h6sQpx7pDAu
BTC balance: 0.0 BTC

ETH address: 0x05eAe4F546be8CE1F42a24f6B8854dBCD52D1081
ETH balance: 0.0 ETH
Seed:ghost either horse make filter core include airport online effort scale rude
BTC a

In [40]:
# Esegui lo script Python
!python /content/EnigmaCracker.py

/bin/sh: 1: gnome-terminal: not found


In [41]:
# Visualizza il contenuto del file di log
!cat /content/enigmacracker.log

In [42]:
# Visualizza il contenuto del file di log per confermare se è vuoto o meno
!cat /content/enigmacracker.log

In [43]:
import os

source_file = '/content/enigmacracker.log'
destination_file = '/content/enigmacracker_copy.log'

try:
    # Read the content of the source file
    with open(source_file, 'r') as f_source:
        log_content = f_source.read()

    # Write the content to the destination file
    with open(destination_file, 'w') as f_dest:
        f_dest.write(log_content)

    print(f"Content of '{source_file}' copied to '{destination_file}' successfully.")

except FileNotFoundError:
    print(f"Error: The file '{source_file}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Content of '/content/enigmacracker.log' copied to '/content/enigmacracker_copy.log' successfully.


In [44]:
# Visualizza il contenuto del file di log
!cat /content/enigmacracker.log